In [1]:
import pyspark.sql.functions as F
import json
from cdk.services.api.energi_data_service import EnergiDataService
from cdk.common_modules.access.secrets import Secrets
from cdk.common_modules.spark.spark_config import SparkConfig
from cdk.common_modules.spark.spark_session_builder import SparkSessionBuilder
from cdk.common_modules.utility.dwh_columns import DwhColumns 

storage_account_name = "adlsthuehomelakehousedev"

# Set Spark configurations
spark_config = SparkConfig(Secrets())

# Add jars to install
spark_config.add_jars_to_install(['hudi', 'azure_storage', 'delta'])

# Add storage account access
spark_config.add_storage_account_access(storage_account_name, method='access_key')

# Build SparkSession
spark = SparkSessionBuilder("ViewAzureData", spark_config).build()


{"asctime": "2023-12-16 17:24:23,301", "levelname": "INFO", "name": "root", "message": "Loading secret ADLS_adlsthuehomelakehousedev_access_key"}
:: loading settings :: url = jar:file:/opt/bitnami/python/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.hudi#hudi-spark3.3-bundle_2.12 added as a dependency
org.apache.hadoop#hadoop-azure added as a dependency
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-50194fb8-ba59-45cc-aea0-f263828cf64c;1.0
	confs: [default]
	found org.apache.hudi#hudi-spark3.3-bundle_2.12;0.13.1 in central
	found org.apache.hadoop#hadoop-azure;3.3.3 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found commons-logging#commons-logging;1.1.3 in central
	found commons-codec#commons-codec;1.15 in central
	found com.microsoft.azure#azure-storage;7.0.1 in central
	found com.fasterxml.jackson.core#jackson-core;2.13.2 in central
	found org.slf4j#slf4j-api;1.7.36 in central
	found com.microsoft.azure#azure-keyvault-core;1.0.0 in central
	found com.google.guava#guava;

23/12/16 17:24:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
dataset_name = "stock_transactions"
dataset_path = f"nordea"#/20231210/{dataset_name}"
container_name = "landing"
destination_container_name = "bronze"

# landing_path = f"hdfs://namenode:9000/data/landing/energi_data_service/{dataset_name}"
landing_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{dataset_path}"
bronze_path = f"abfss://{destination_container_name}@{storage_account_name}.dfs.core.windows.net/delta/{dataset_path}/{dataset_name}"



In [4]:
import datetime

# from cdk.common_modules.utility import dwh_columns
from cdk.common_modules.utility.dwh_columns import DwhColumns 

df = (
    spark.read.csv(landing_path, header=True, sep=";")
    # .withColumn("date", F.to_date(F.concat_ws('-','year','month','day'), "yyyy-MM-dd"))
    # .filter((F.col("date")>datetime.date(2023,12,9)))
)

# df.show()

dwh_columns = DwhColumns()

df_filename = dwh_columns.add_filename(df)

df_timestamp =dwh_columns.add_updated_timestamp(df)

df_filename.show(truncate=False)

{"asctime": "2023-12-16 13:38:45,728", "levelname": "INFO", "name": "cdk.common_modules.utility.logging", "message": "Running add_filename with args: (<cdk.common_modules.utility.dwh_columns.DwhColumns object at 0x7f52f481c640>, DataFrame[Status: string, Instrument navn: string, Opdateret den: string, Transaktionstype: string, Antal: string, Total: string, _c6: string, year: int, month: int, day: int]) and kwargs: {}"}
{"asctime": "2023-12-16 13:38:45,783", "levelname": "INFO", "name": "cdk.common_modules.utility.logging", "message": "Finished running add_filename"}
{"asctime": "2023-12-16 13:38:45,786", "levelname": "INFO", "name": "cdk.common_modules.utility.logging", "message": "Running add_updated_timestamp with args: (<cdk.common_modules.utility.dwh_columns.DwhColumns object at 0x7f52f481c640>, DataFrame[Status: string, Instrument navn: string, Opdateret den: string, Transaktionstype: string, Antal: string, Total: string, _c6: string, year: int, month: int, day: int]) and kwargs: 

In [18]:
from pyspark.sql.utils import AnalysisException


try:
    df_temp = (
        spark.read.format("delta") 
            .option("readChangeFeed", "true") 
            .option("startingTimestamp", '2023-12-17 05:45:46') 
            .load(bronze_path)
    )
    df_temp.show()
except AnalysisException as e:
    if "Please use a timestamp before or at" in str(e):
        print("No new data")
    else:
        raise e
    # df_temp = df

No new data


In [13]:
from delta.tables import DeltaTable
bronze_path_tmp = f"abfss://{destination_container_name}@{storage_account_name}.dfs.core.windows.net/delta/{dataset_path}/test_stock_transactions"
df = (
    spark.read.format('delta').load(bronze_path)
    # .withColumn("date", F.to_date(F.concat_ws('-','year','month','day'), "yyyy-MM-dd"))
    # .filter((F.col("date")>datetime.date(2023,12,9)))
)
# df.show()
(
    DeltaTable
        .createIfNotExists(spark)
        # .tableName(file.name)
        .addColumns(df.schema)
        .location(bronze_path_tmp)
        .property("delta.enableChangeDataFeed", "true").execute()
)

In [12]:
(
    df
        .write
        .format('delta')
        .mode('append')
        .save(bronze_path_tmp)

)